# Irina Viner-Usmanova Rhythmic Gymnastics Centre

The project presented in this notebook is an adaptation of the Irina Viner-Usmanova Rhythmic Gymnastics Centre, originally designed by CPU Pride for Moscow, Russia. Its most prominent feature is the wavy roof-structure, inspired by the movement of a rhythmic gymnastic’s strip. The program elaborated here is a personal interpretation of the original design, developed in a 5 day collaborative sprint. This notebook uses the __Julia__ programming language and the __Khepri__ algorithmic design tool to model the building's geometry.

<img src="./Renders/light.png" width="900"> 

In [ ]:
using WebIO

In [ ]:
using Interact

In [ ]:
using Khepri

In [ ]:
render_size(800, 400) # Khepri backends size

Avoid tests macro: `true` to skip tests, and `false` to run them.

In [ ]:
avoid_tests = Parameter(true)

macro test(expr...)
  quote
    if !avoid_tests() 
        begin
            $(esc(expr...))
        end
    end
  end
end

In [ ]:
# avoid_tests(false)

## Sinusoidal Curves

 | Sinunoisal wave parameters | Other sinusoidal curves |
 |----------------------------|-------------------------|
 |<img src="./Figures/sin_wave.png" width="480"> | <img src="./Figures/sin_curves.png" width="350">|

`sinusoidal` parameters explained:
* a is the amplitude
* omega is the frequency
* fi is the phase

In [ ]:
sinusoidal(a, omega, fi, x) = a*sin(omega*x+fi)

In [ ]:
sin_array_y(p, a, omega, fi, dist, n) = [p+vxy(i, sinusoidal(a, omega, fi, i)) for i in division(0, dist, n)]

In [ ]:
@test begin
    backend(notebook)
    new_backend()
    line(sin_array_y(u0(), 5, 1, 0, 50, 100))
end

Expected result:
<img src="./Plots/sin_array_y.png" width="600">

`damped_sin_wave`'s parameters explained
* a is the initial amplitude (the highest peak)
* b is the decay constant
* omega is the angular frequency

In [ ]:
damped_sin_wave(a, d, omega, x) = a*exp(-(d*x))*sin(omega*x)

In [ ]:
damped_sin_array_z(p, a, d, omega, dist, n) = [p+vxz(i, damped_sin_wave(a, d, omega, i)) for i in division(0, dist, n)]

In [ ]:
@test begin
    backend(notebook)
    new_backend()
    line(damped_sin_array_z(u0(), 5, 0.1, 1, 50, 100))
end

Expected result:
<img src="./Plots/damped_sin_array.png" width="600">

In [ ]:
damped_sin_roof_pts(p, a_x, a_y, fi, decay, om_x, om_y, dist_x, dist_y, n_x, n_y) =
    map_division((x, y)->p+vxyz(x, y, sinusoidal(a_x, om_x, fi, x)+damped_sin_wave(a_y, decay, om_y, y)),
        0, dist_x, n_x,
        0, dist_y, n_y)

In [ ]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampx = widget(0:0.1:3, label="Amplitude x"),
                ampy = widget(0:0.1:20, label="Amplitude y")
            delete_all_shapes()
            surface_grid(damped_sin_roof_pts(u0(), ampx, ampy, 0, 0.1, 0.2, 1, 50, 100, 100, 200))
    end
end

Expected result:
<img src="./Plots/damped_surface.png" width="600">

In [ ]:
damped_sin_roof_pts(p, h, a_x, a_y_min, a_y_max, fi, decay, om_x, om_y, dist_x, dist_y, n_x, n_y) =
    map_division((x, y) ->
                y <= d_i ?
                p + vxyz(x, 
                         -sin(y/d_i*(1*pi)), 
                         y*h/d_i + sin(x/dist_x*pi)sinusoidal(a_x, om_x, fi-y*pi/dist_y, x)*(y*a_x/d_i)) :
                p + vxyz(x,
                         y,
                         h + sin(x/dist_x*pi)*sinusoidal(a_x, om_x, fi-y*pi/dist_y, x) +
                         damped_sin_wave(a_y_max - (a_y_max-a_y_min)/dist_x*x, decay, om_y, y)),
             0, dist_x, n_x,
             0, dist_y, n_y)

`d_i`is the distance between the pavillion starting point and the beginning of the dumped sine curve.

In [ ]:
@test begin
    backend(meshcat)
    new_backend()
    d_i = 0.5
    @manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                    ampy_min = widget(1:0.1:15, label="Minimum Amplitude y"),
                    ampy_max = widget(1:0.1:15, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
                surface_grid(damped_sin_roof_pts(u0(), h, ampx, ampy_min, ampy_max, pi, 0.03, pi/50, pi/10, 60, 100, 120, 800))
    end
end

Expected result:
<img src="./Plots/roof_surface.png" width="600">

## Pavilion Dimensions

 | Roof dimentions explained | `d_i` explained |
 |---------------------------|-----------------|
 |<img src="./Figures/roof_dims.jpg" width="400"> | <img src="./Figures/roof_di.jpg" width="400"> |

__Double sinuoid parameters:__

* amp_x = amplitude of the sinusoid along the x axis
* amp_y_min_top = minimum amplitude of the damped sinusoid along the y axis for the top layer of the roof
* amp_y_max_top = maximum amplitude of the damped sinusoid along the y axis for the top layer of the roof
* amp_y_max_bottom = maximum amplitude of the damped sinusoid along the y axis for the bottom layer of the roof
* amp_y_min_bottom = minimum amplitude of the damped sinusoid along the y axis for the bottom layer of the roof
* fi = sinusoid's phase along the x axis
* decay = damped sinusoid's decay along the y axis
* om_x = frequency of the sinusoid in x
* om_y = frequency of the damped sinusoid in y

__Genaral pavilion parameters:__

* pav_width = pavilion's width (along x axis)
* pav_length = pavilion's lenght (along y axis)
* pav_height = pavilion's height (z axis)
* d_width = distance between roof layers in the x axis (top layer is wider)
* d_length = distance between roof layers in the y axis (top layer is longer)
* d_height = distance between roof layers in the z axis (height of the roof truss)
* d_i = distance before the damped sinusoid starts shaping the roof

__Construction elements parameters:__

* glass_panel_height = façade glass panel's height (width is defined by the number of points in the `damped_sinusoid` array)
* n_panels_y = number of row sobdivisions for triangular roof panels and glass wall vertical lines in length
* n_panels_x = number of row sobdivisions for triangular roof panels and glass wall vertical lines in width
* n_glass_verts = number of vertical panels per glass line
* pinwheel_rec_level = number of times the pattern gets recursively subdivided
* pav_slab_thickness = floor slab thickness
* pav_struct_thickness = inner structural wall's thickness
* n_floors = number of floors inside the pavilion
* n_wall_in_width = number of transverse walls on each floor
* n_wall_in_length = number of longitudinal walls on each floor

In [ ]:
amp_x = 2.5
amp_y_min_top = 4
amp_y_max_top = 11
amp_y_max_bottom = 10
amp_y_min_bottom = 3
fi = pi
decay = 0.03

pav_width = 60
pav_length = 100
pav_height = 14
d_width = 1.5
d_length = 1
d_height = 3
d_i = d_length

om_x = pi/pav_width
om_y = 10*pi/pav_length
om_y_bottom = 10*pi/(pav_length - 2*d_length)

glass_panel_height = 1
n_panels_y = 100 
n_panels_x = 50
n_glass_verts = 10
pinwheel_rec_level = 3
pav_slab_thickness = 0.5
pav_struct_thickness = 0.3
n_floors = 3
n_wall_in_width = 4
n_wall_in_length = 6

## Roof Truss

### Basic truss elements

The following functions create truss elements:
* `free_node` and `fixed_node`create truss nodes
* `bar`creates truss bars
* `panel`creates truss panels

CAD version of truss elements:

In [ ]:
free_node(pt) = sphere(pt, 0.1)
fixed_node(pt) = sphere(pt, 0.1)

In [ ]:
truss_panel(ps) = surface_polygon(ps)

In [ ]:
e agora? skype?

BIM families for truss elements:

In [ ]:
free_node_fam = truss_node_family_element(default_truss_node_family(), support=Khepri.truss_node_support(), radius=0.1)
sup_node_fam = truss_node_family_element(default_truss_node_family(), support=Khepri.truss_node_support(ux=true, uy=true, uz=true), radius=0.1);

BIM version of truss elements:

In [ ]:
fixed_node(p) = truss_node(p, family=sup_node_fam)
free_node(p) = truss_node(p, family=free_node_fam)

In [ ]:
bar(p, q) = truss_bar(p, q)

### Spatial truss

`truss_ptss`receives two sets of points, the bottom and top points defining the shape of the truss to create.
It returns the points defining the truss structure.

In [ ]:
truss_ptss(bottom_ptss,top_ptss) =
    length(top_ptss) == length(bottom_ptss) ?
    vcat([[pts,qts] for (pts,qts) in zip(bottom_ptss,top_ptss)]...) :
    vcat([[pts,qts] for (pts,qts) in zip(top_ptss[1:end-1],bottom_ptss)]..., [top_ptss[end]])

`truss`receives the type of truss to create, as well as both bottom and top points defining its shape.
It creates a 3D spatial truss.

In [ ]:
truss(truss_type; bottom_ptss=planar(p=x(0)), top_ptss=planar(p=z(1))) =
    let ptss = truss_ptss(bottom_ptss, top_ptss)
      truss_type(ptss)
  end

 | Vierendeel modular block front | Vierendeel modular block side |
 |----------------------------|-------------------------|
 |<img src="./Figures/front_vierendeel.png" width="350"> | <img src="./Figures/side_vierendeel.png" width="350">|

`vierendeel`creates a vierendeel truss structure.

In [ ]:
vierendeel(ptss, first=true) =
    let ais = ptss[1],
        bis = ptss[2],
        cis = ptss[3],
        dis = ptss[4]
      (first ? fixed_node : free_node).(ais)
      free_node.(bis)
      bar.(ais, bis)
      bar.(ais, cis)
      bar.(bis, dis)
      bar.(ais[2:end], ais[1:end-1])
      bar.(bis[2:end], bis[1:end-1])
      if ptss[5:end] == []
          fixed_node.(cis)
          free_node.(dis)
          bar.(cis[2:end], cis[1:end-1])
          bar.(dis[2:end], dis[1:end-1])
          bar.(dis, cis)
      else
          vierendeel(ptss[3:end], false)
      end
    end

In [ ]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                    ampy_top_min = widget(3:0.1:15, label="Minimum Amplitude y"),
                    ampy_top_max = widget(3:0.1:15, label="Maximum Amplitude y"),
                    ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                    ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
        truss(vierendeel, 
          top_ptss=damped_sin_roof_pts(u0(), pav_height,
                                          ampx, ampy_top_min, ampy_top_max,
                                          fi, decay, om_x, om_y,
                                          pav_width, pav_length-d_length, 50, 80),
          bottom_ptss=damped_sin_roof_pts(xy(d_width,d_length), pav_height-d_height,
                                         ampx, ampy_bottom_min, ampy_bottom_max,
                                         fi, decay, om_x, om_y_bottom,
                                         pav_width - d_width*2, pav_length-2*d_length, 50, 80))
    end
end

Expected result:
<img src="./Plots/truss_roof_meshcat.png" width="800">

In [ ]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    truss(vierendeel, 
      top_ptss=damped_sin_roof_pts(u0(), pav_height,
                                      amp_x, amp_y_min_top, amp_y_max_top,
                                      fi, decay, om_x, om_y,
                                      pav_width, pav_length-d_length, 5, 8),
      bottom_ptss=damped_sin_roof_pts(xy(d_width,d_length), pav_height-d_height,
                                     amp_x, amp_y_min_bottom, amp_y_max_bottom,
                                     fi, decay, om_x, om_y_bottom,
                                     pav_width - d_width*2, pav_length-2*d_length, 5, 8))
#     render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
#     render_view("truss_roof_unity")
end

Expected result:
<img src="./Plots/truss_roof_unity.png" width="800">

## Roof Surface

### Auxiliar functions

`tranpose_array`receives an array of arrays and transposes it.

In [ ]:
transpose_array(arrays) =
    [[row[i] for row in arrays]
     for i in 1:length(arrays[1])]

`pts_distances`receives an array of points and returns an array with the distances between ordered points.

In [ ]:
pts_distances(pts, last_pt=true) =
    let n =  last_pt ? 0 : 1
      [distance(p,q)
       for (p,q) in zip(pts,[pts[2:end]...,pts[1]])][1:end-n]
       end

In [ ]:
shape_grid_polygon_vertices(pts) =
    () -> pts

`itera_2triangs`receives an array of arrays of points. It returns the same points rearranged to create a triangular grid of points.

In [ ]:
itera_2triangs(ptss) =
    vcat([vcat([[[p0,p1,p3],[p1,p2,p3]]
                for (p0,p1,p2,p3)
                in zip(pts0[1:end-1], pts1[1:end-1], pts1[2:end], pts0[2:end])]...)
         for (pts0, pts1) in zip(ptss[1:end-1], ptss[2:end])]...)

BIM families for roof panels:

In [ ]:
roof_panel_fam = panel_family_element(default_panel_family())
yellow_panel_fam = panel_family_element(default_panel_family());

### Roof option 1: simple surfaces

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()
    top_roof = damped_sin_roof_pts(x(0), 15,
                                   3, 6, 9,
                                   fi, decay, om_x, om_y,
                                   pav_width, pav_length-d_length, 50, 100)
    bottom_roof = damped_sin_roof_pts(xy(d_width,d_length*1), 15-d_height,
                                      3, 6, 8,
                                      fi, decay, om_x, om_y_bottom,
                                      pav_width - d_width*2, pav_length-2*d_length, 50, 100)
    surface_grid(top_roof)
    surface_grid(bottom_roof)     
end

Expected result:
<img src="./Plots/roof_surf_grid.png" width="800">

### Roof option 2: triangular roof panels

CAD version: using `surface_polygon` subdivision

In [ ]:
@test begin
    backend(autocad)
    new_backend()
    map(ps->surface_polygon(ps), itera_2triangs(top_roof))
    map(ps->surface_polygon(ps), itera_2triangs(bottom_roof))        
end

Expected result:
<img src="./Plots/roof_surf_polygons_cad.png" width="800">

BIM version: using `panel` subdivision

In [ ]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    map(ps->panel(ps, family=roof_panel_fam), itera_2triangs(top_roof))
    map(ps->panel(ps, family=roof_panel_fam), itera_2triangs(bottom_roof))        
end

In [ ]:
# render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
# render_view("roof_panels_unity")

Expected result:
<img src="./Plots/roof_panels_unity.png" width="800">

### Roof lateral edges option 1: simple triangular panels

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()
    map(ps->surface_polygon(ps), itera_2triangs([top_roof[1],bottom_roof[1]]))
    map(ps->surface_polygon(ps), itera_2triangs([top_roof[end],bottom_roof[end]]))
    map(ps->surface_polygon(ps), itera_2triangs([transpose_array(top_roof)[end], transpose_array(bottom_roof)[end]]))
end

Expected result:
<img src="./Plots/triangle_roof_side.png" width="800">

### Roof lateral edges option 2: Pinwheel tiling panels

`pattern`creates different patterns on parametric surfaces

In [ ]:
pattern(fshape, pts...; args...) =
  fshape(pts...)(; (k=>v(pts) for (k,v) in args)...)

`pinwheel_rule`defines the basic subdivision rule of the pinwheel tiling.

 |    Pinwheel rule Tiling   |
 |---------------------------|
 |<img src="./Figures/tiling.png" width="400"> |
 source: https://mathematica.stackexchange.com/questions/55677/how-to-generate-nonperiodic-tilings

In [ ]:
pinwheel_rule(pts, ratio1=0.383) =
    let dists = pts_distances(pts)
        max_dist = maximum(dists)
        min_dist = minimum(dists)
        ratio2 = 1 - ratio1
     if max_dist == dists[1]
         let pm = intermediate_loc(pts[1], pts[2], min_dist==dists[2] ? ratio1 : ratio2)
           [[pts[1],pm,pts[3]],
            [pts[2],pm,pts[3]]]
         end
     elseif max_dist == dists[2]
         let pm = intermediate_loc(pts[2], pts[3], min_dist==dists[1] ? ratio1 : ratio2)
           [[pts[1],pm,pts[2]],
            [pts[1],pm,pts[3]]]
         end
     else
        let pm = intermediate_loc(pts[1], pts[3], min_dist==dists[1] ? ratio1 : ratio2)
          [[pts[1],pm,pts[2]],
           [pts[2],pm,pts[3]]]
        end
     end
  end

`pinwheel_recursive_rule` repeats the basic subdivision rule `pinwheel_rule` several times (value set by the parameter `level`).

In [ ]:
pinwheel_recursive_rule(pts, ratio1=0.383, level=1) =
    let qts = pinwheel_rule(pts, ratio1)
      level == 1 ?
      qts :
      [qts...,
       pinwheel_recursive_rule(qts[1], ratio1, level-1)...,
       pinwheel_recursive_rule(qts[2], ratio1, level-1)...]
     end

`pinwheel_tiling`receives a surface (described by its points) and creates the pinwheel tiling on it.

In [ ]:
pinwheel_tiling(pts; ratio1=0.383, level=1) =
        map(surface_polygon, pinwheel_recursive_rule(pts, ratio1, level))

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()
    top_roof = damped_sin_roof_pts(x(0), 15,
                                   3, 6, 9,
                                   fi, decay, om_x, om_y,
                                   pav_width, pav_length-d_length, 50, 100)
    bottom_roof = damped_sin_roof_pts(xy(d_width,d_length*1), 15-d_height,
                                      3, 6, 8,
                                      fi, decay, om_x, om_y_bottom,
                                      pav_width - d_width*2, pav_length-2*d_length, 50, 100)
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([top_roof[1],bottom_roof[1]])),
                     level=3)
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([top_roof[end],bottom_roof[end]])),
                     level=3)
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([transpose_array(top_roof)[end],transpose_array(bottom_roof)[end]])),
                     level=3)
end

|  | Expected result: |
|-----------------------------------------------------------|-----------------------------------------------------------|
| <img src="./Plots/pinwheel_roof_side.png" width="700"> | <img src="./Plots/pinwheel_closeup.png" width="300"> |

### Roof lateral edges option 2: BIM version

In [ ]:
pinwheel_tiling(pts; ratio1=0.383, level=1) =
        map(pts -> panel(pts, family=yellow_panel_fam), pinwheel_recursive_rule(pts, ratio1, level))

In [ ]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    top_roof = damped_sin_roof_pts(x(0), 15,
                                   3, 6, 9,
                                   fi, decay, om_x, om_y,
                                   pav_width, pav_length-d_length, 50, 100)
    bottom_roof = damped_sin_roof_pts(xy(d_width,d_length*1), 15-d_height,
                                      3, 6, 8,
                                      fi, decay, om_x, om_y_bottom,
                                      pav_width - d_width*2, pav_length-2*d_length, 50, 100)
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([top_roof[1],bottom_roof[1]])),
                     level=3)
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([top_roof[end],bottom_roof[end]])),
                     level=3)
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([transpose_array(top_roof)[end],transpose_array(bottom_roof)[end]])),
                     level=3)
end

In [ ]:
# render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
# render_view("side_roof_panels_unity1")

|  | Expected result: |
|-----------------------------------------------------------|-----------------------------------------------------------|
| <img src="./Plots/side_roof_panels_unity1.png" width="450"> | <img src="./Plots/side_roof_panels_unity2.png" width="450"> |

## Complete Roof structure

Abstracting `pinwheel_tiling` function for the roof sides:

In [ ]:
pinwhell_broad(pts1, pts2, rec_level) = 
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices, itera_2triangs([pts1,pts2])), level=rec_level)

Roof point matices. `t_diff` defines the distance between the roof surface and the truss structure within. It's currently set to 20 cm:

In [ ]:
top_roof_ptss(p, n_x, n_y, truss)=
    let dist_y = pav_length-d_length
        t_diff = truss ? 0.2 : 0
        damped_sin_roof_pts(p, pav_height-t_diff,
                            amp_x, amp_y_min_top, amp_y_max_top,
                            fi, decay, om_x, om_y,
                            pav_width, dist_y, n_x, n_y)
    end

bottom_roof_pts(p, n_x, n_y, truss)=
    let dist_y = pav_length-d_length
         t_diff = truss ? 0.2 : 0
        damped_sin_roof_pts(p+vxy(d_width,d_length*1), pav_height-d_height+t_diff,
                            amp_x, amp_y_min_top, amp_y_max_top,
                            fi, decay, om_x, om_y_bottom,
                            pav_width - d_width*2, pav_length-2*d_length,  n_x, n_y)
    end

Complete roof surface with top, bottom, and side panels:

In [ ]:
roof_surf(p, n_x, n_y)=
    let lev = pinwheel_rec_level
        top_roof = top_roof_ptss(p, n_x, n_y, false)
        bottom_roof = bottom_roof_pts(p, n_x, n_y, false)
        # top and bottom aluminum roof tiles
        map(ps->panel(ps, family=roof_panel_fam), itera_2triangs(top_roof))
        map(ps->panel(ps, family=roof_panel_fam), itera_2triangs(bottom_roof))
        # side yellow copper roof tiles
        pinwhell_broad(top_roof[1], bottom_roof[1], lev)
        pinwhell_broad(top_roof[end], bottom_roof[end], lev)
        pinwhell_broad(transpose_array(top_roof)[end], transpose_array(bottom_roof)[end], lev)
    end

In [ ]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    roof_surf(u0(), 50, 100)
end

In [ ]:
# render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
# render_view("complete_roof_panels_unity")

Expected result:
<img src="./Plots/complete_roof_panels_unity.png" width="800">

Truss sturcture function using the new point matices for the pavilion:

In [ ]:
roof_truss(p, n_x, n_y)=
    let top_roof = top_roof_ptss(p, n_x, n_y, true)
        bottom_roof = bottom_roof_pts(p, n_x, n_y, true)
        truss(vierendeel, top_ptss=top_roof, bottom_ptss=bottom_roof)
    end

In [ ]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    set_backend_family(yellow_panel_fam, unity, unity_material_family("Default/Materials/Glass")) 
    roof_surf(u0(), 50, 100)
    roof_truss(u0(), 30, 70)
end

In [ ]:
# render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
# render_view("complete_roof_unity")

Expected result:
<img src="./Plots/complete_roof_unity.png" width="800">

## Glass Façade

`splines4surf`maps splines over a matrix of points, to visualise surfaces in the notebook backend faster.

In [ ]:
function splines4surf(mtx)
    [spline(pts) for pts in mtx]
    [spline(pts) for pts in transpose_array(mtx)]
end

`damped_sin_glass_wall`creates the pavillion's side glass wall matrixes

In [ ]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z) =
    map_division((y, z) ->
                    y <= d_i ?
                    p + vyz(y, -sin(y/d_i*(1*pi))) :
                    p + vyz(y,
                            z + (z/dist_z)*damped_sin_wave(a_y, decay, om_y, y)),
             0, dist_y, n_y,
             0, dist_z, n_z)

In [ ]:
@test begin
    backend(notebook)
    new_backend()  
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            pav_length-2*d_length, pav_height-d_height, 20, 5)
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            pav_length-2*d_length, pav_height-d_height, 20, 5)
    splines4surf(west_glass_wall)
end

Expected result:
<img src="./Plots/glass_lines.png" width="800">

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()  
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            pav_length-2*d_length, pav_height-d_height, 40, 10)
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            pav_length-2*d_length, pav_height-d_height, 40, 10)
    splines4surf(west_glass_wall)
    splines4surf(east_glass_wall)
end

Expected result:
<img src="./Plots/glass_lines_cad2.png" width="800">

### Glass option 1: surface polygons

`itera_sweeked`creates the pavillion glass walls

In [ ]:
itera_sweeked(ptss) =
    vcat([[let p23=intermediate_loc(p2, p3),
               p24=intermediate_loc(p2, p4)
             [p0,p1,p24,p23]
           end
          for (p0,p1,p2,p3,p4)
          in zip(pts0[1:end-1], pts1[1:end-1], pts1[2:end], pts0[2:end], pts2[2:end])]
         for (pts0, pts1, pts2)
         in zip(ptss[1:end-2], ptss[2:end-1],ptss[3:end])]...)

In [ ]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                    ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
                west_glass_wall = damped_sin_glass_wall(x(d_width),
                                                        ampy_bottom_max,
                                                        fi, decay, om_y,
                                                        pav_length-2*d_length, h-d_height, 85, 20)
                east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                                        ampy_bottom_min,
                                                        fi, decay, om_y,
                                                        pav_length-2*d_length, h-d_height, 85, 20)
                map(ps->surface_polygon(ps), itera_sweeked(west_glass_wall))
                map(ps->surface_polygon(ps), itera_sweeked(east_glass_wall))
    end
end

Expected result:
<img src="./Plots/glass_polys2.png" width="800">

### Glass option 2: vertical BIM panels of diferent sizes

Given a list of points (closed polygon vertices), this function creates a polygonal glass panel surrounded by a thin metal framing all around:

In [ ]:
framed_panel(pts)=
    begin
        panel(pts, family=default_panel_family())
        for (p,q) in zip(pts, [pts[2:end]...,pts[1]])
            free_column(p,q, family=frame_fam)
        end
    end

Frame BIM family

In [ ]:
frame_width=0.1
frame_fam = column_family_element(default_column_family(), profile=rectangular_profile(frame_width, frame_width))

In [ ]:
@test begin

    backend(meshcat)
    new_backend()
    @manipulate for radius = widget(1:1:5, label="Radius"),
                    sides = widget(1:1:10, label="Sides"),
                    angle = widget(1:1:20, label="Angle")
                    delete_all_shapes()        
                    framed_panel(regular_polygon_vertices(sides, x(-10), radius, angle))
                    framed_panel(regular_polygon_vertices(sides+2, x(0), radius, angle))
                    framed_panel(regular_polygon_vertices(sides+5, x(10), radius, angle))
                    end
  
end

Expected result:
<img src="./Plots/framed_panel.png" width="800">

Creates a vertical line of `framed_panels` given a list of points in z:

In [ ]:
vertical_panel_line(pts, v, panel_width)=
    let pav_panel(p, q) = framed_panel([p, p+v*panel_width, q+v*panel_width, q])
        [pav_panel(p, q) for (p, q) in zip(pts[1:end-1], pts[2:end])]
    end

Wall points correction:

In [ ]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z) =
    map_division((y, z) ->
                    y <= d_i ?
                    p + vyz(-sin(y/d_i*(1*pi)), z) :
                    p + vyz(y,
                            z + (z/dist_z)*damped_sin_wave(a_y, decay, om_y, y)),
             0, dist_y, n_y,
             0, dist_z, n_z)

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()  
    west_glass_wall = damped_sin_glass_wall(xy(d_width, d_length),
                                            amp_y_max_top,
                                            fi, decay, om_y_bottom,
                                            pav_length-2*d_length, pav_height-d_height, 40, 10)
    east_glass_wall = damped_sin_glass_wall(xy(pav_width-d_width, d_length),
                                            amp_y_min_bottom,
                                            fi, decay, om_y_bottom,
                                            pav_length-2*d_length, pav_height-d_height, 40, 10)
    splines4surf(west_glass_wall)
    splines4surf(east_glass_wall)
#     surface_grid(bottom_roof_pts(u0(), 40, 100, false))
end

Expected result:
<img src="./Plots/glass_lines_cad.png" width="800">

In [ ]:
@test begin
    backend(autocad)
    n_y = 40
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    panel_width = dist_y/n_y
    vertical_panel_line(west_glass_wall[1], vy(1), panel_width)
end

Expected result:
<img src="./Plots/first_row_panels.png" width="800">

Same function, but vector `v` now already considers the panel width, it's not a normal vector anymore:

In [ ]:
vertical_panel_line(pts, v)=
    let pav_panel(p, q) = framed_panel([p, p+v, q+v, q])
        [pav_panel(p, q) for (p, q) in zip(pts[1:end-1], pts[2:end])]
    end

Makes a `vertical_panel_line` for all lines in the matrix:

In [ ]:
vertical_panel_lines(mtx)=
    let v = mtx[2][1]-mtx[1][1]
        [vertical_panel_line(pts, v) for pts in mtx]
    end

In [ ]:
@test begin
    backend(autocad)
    n_y = 40
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    delete_all_shapes()
    vertical_panel_lines(west_glass_wall)
end

Expected result:
<img src="./Plots/weast_facade_panel.png" width="800">

### Glass option 3: same size vertical BIM panels

Predicate functions: is current `z` is above the limit ?

In [ ]:
is_above_z_limit(z, z_lim)= z > z_lim

`vert_pts` creates the orgin points for all vertical panels above p. Stops creating points in z when the set limit is reached:

In [ ]:
vert_pts(p, panel_height, z_lim) = is_above_z_limit(p.z, z_lim) ? [p] : [p, vert_pts(p+vz(panel_height), panel_height, z_lim)...]

`damped_sin_glass_wall` creates a vertical line of panels for all points between `p` and `p+vy(dist_y)`. The `z_limit` for all vertical lines of panels is defined by the`z` value of the `damped_sin_wave` at any moment in the `y` progression.

In [ ]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z, panel_height) =
    let pts = map_division(y -> p+vy(y), 0, dist_y, n_y)
    zs = map_division(y -> dist_z+damped_sin_wave(a_y, decay, om_y, y), 0, dist_y, n_y) 
    [vert_pts(p, panel_height, z_lim) for (p, z_lim) in zip(pts, zs)]
    end

 `glass_panel_height` defines façade glass panel's height (width is defined by the number of points in the `damped_sinusoid` array)

In [ ]:
@test begin
    backend(autocad)
    glass_panel_height = 0.5
    n_y = 40
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z, glass_panel_height)
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z, glass_panel_height)
    delete_all_shapes()
    vertical_panel_lines(west_glass_wall)
end

Expected result:
<img src="./Plots/planels_2d_try_cad.png" width="800">

### Glass option 4: intercalated rows of vertical BIM panels

Diffrent starting point for vertical panels in the odd case:

In [ ]:
vert_pts_odd(p, panel_height, z_lim) = [p, vert_pts(p+vz(panel_height/2), panel_height, z_lim)...]

Interweaving 2 different functions depending on `count` being pair or odd:

In [ ]:
f_weave(f1, f2, count) = isodd(count) ? f1 : f2

Interweaved version of the `damped_sin_glass_wall` function. Different starting height for the first panel of odd rows.

In [ ]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z, panel_height) =
    let pts = map_division(y -> p+vy(y), 0, dist_y, n_y)
        zs = map_division(y -> dist_z+damped_sin_wave(a_y, decay, om_y, y), 0, dist_y, n_y) 
        ns = 1:length(zs)
     [f_weave(vert_pts_odd(p, panel_height, z_lim),vert_pts(p, panel_height, z_lim), count) for (p, z_lim, count) in zip(pts, zs, ns)]
    end

In [ ]:
@test begin
    backend(autocad)
    glass_panel_height = 1
    n_y = 40
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z, glass_panel_height)
    delete_all_shapes()
    vertical_panel_lines(west_glass_wall)
end

Expected result:
<img src="./Plots/panels_weave.png" width="800">

`vertical_panel_line` corrections to match roof wave:

In [ ]:
vertical_panel_line_centred(pts, v)=
    let pav_panel(p, q) = framed_panel([p, p+v, q+v, q])
        [pav_panel(p-v/2, q-v/2) for (p, q) in zip(pts[1:end-1], pts[2:end])]
    end

In [ ]:
centred_vertical_panel_lines(mtx)=
    let v = mtx[2][1]-mtx[1][1]
        [vertical_panel_line_centred(pts, v) for pts in mtx]
    end

In [ ]:
@test begin
    backend(autocad)
    delete_all_shapes()
    west_glass_wall_i = damped_sin_glass_wall(xy(d_width, d_length),
                                                amp_y_max_top,
                                                fi, decay,  om_y_bottom,
                                                dist_y, dist_z, n_y, n_z)
    west_glass_wall_r = damped_sin_glass_wall(xy(d_width, d_length),
                                                amp_y_max_top,
                                                fi, decay, om_y_bottom,
                                                dist_y, dist_z, n_y, n_z, glass_panel_height)
    splines4surf(west_glass_wall_i)
    centred_vertical_panel_lines(west_glass_wall_r)
end

Expected result:
<img src="./Plots/wall_panel_comparison.png" width="800">

In [ ]:
@test begin
    backend(unity)
    delete_all_shapes()
    n_y = 100
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    west_glass_wall = damped_sin_glass_wall(xy(d_width, d_length),
                                            amp_y_max_top,
                                            fi, decay, om_y_bottom,
                                            dist_y, dist_z, n_y, n_z, glass_panel_height)
    east_glass_wall = damped_sin_glass_wall(xy(pav_width-d_width, d_length),
                                            amp_y_min_bottom,
                                            fi, decay, om_y_bottom,
                                            dist_y, dist_z, n_y, n_z, glass_panel_height)
    ground()
    roof_surf(u0(), 50, 100)
    centred_vertical_panel_lines(west_glass_wall)
    centred_vertical_panel_lines(east_glass_wall)
end

In [ ]:
# render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
# render_view("roof_side_walls_unity")

Expected result:
<img src="./Plots/roof_side_walls_unity.png" width="800">

### Adding front glass wall

In [ ]:
damped_sin_front_wall(p, a_x, fi, decay, om_x, dist_x, dist_z, n_x, n_z, panel_height) =
    let pts = map_division(x -> p+vx(x), 0, dist_x, n_x)
        zs = map_division(x -> dist_z+sin(x/dist_x*pi)*sinusoidal(a_x, om_x, fi-pi, x), 0, dist_x, n_x)
        ns = 1:length(zs)
     [f_weave(vert_pts_odd(p, panel_height, z_lim),vert_pts(p, panel_height, z_lim), count) for (p, z_lim, count) in zip(pts, zs, ns)]
    end

This function joins all glass walls in the pavilion (sides and front):

In [ ]:
pav_walls(p, n_x, n_y, n_z)=
    let dist_y = pav_length-2*d_length
        dist_z = pav_height-d_height
        dist_x = pav_width-d_width*2
        west_glass_wall = damped_sin_glass_wall(p+vxy(d_width, d_length),
                                            amp_y_max_top,
                                            fi, decay, om_y_bottom,
                                            dist_y, dist_z, n_y, n_z, glass_panel_height)
        east_glass_wall = damped_sin_glass_wall(p+vxy(pav_width-d_width, d_length),
                                            amp_y_min_bottom,
                                            fi, decay, om_y_bottom,
                                            dist_y, dist_z, n_y, n_z, glass_panel_height)       
        panel_w = norm(west_glass_wall[2][1]-west_glass_wall[1][1])
        north_glass_wall = damped_sin_front_wall(p+vxy(d_width, dist_y+panel_w/2), 
                                            amp_x, fi, decay, om_x, 
                                            dist_x, dist_z, n_x, n_z, glass_panel_height)
    centred_vertical_panel_lines(west_glass_wall[1:end-1])
    centred_vertical_panel_lines(east_glass_wall[1:end-1])
    vertical_panel_lines(north_glass_wall[1:end-1])
end

In [ ]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    roof_surf(u0(), 50, n_panels_y)
    pav_walls(u0(), 50, n_panels_y, n_glass_verts)
    set_view(xyz(29.727153778076172,163.0078582763672,10.998817443847656), xyz(29.728837966918945,162.0078582763672,10.998811721801758), 35.0)
#     render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
#     render_view("all_walls_unity")
end

Expected result:
<img src="./Plots/all_walls_unity.png" width="800">

## Interior Plan

BIM families for walls and slabs:

In [ ]:
pav_slab_fam = slab_family_element(default_slab_family(), thickness=pav_slab_thickness);

In [ ]:
pav_wall_struct_fam = wall_family_element(default_wall_family(), thickness=pav_struct_thickness);

`pav_slabs` creates the main slabs

In [ ]:
pav_slabs(p, length, height, width, n_floors)=
    for i in division(0, height-2, n_floors, false)
        slab(rectangular_path(p, width, length), level=i, family=pav_slab_fam)
    end

In [ ]:
@test begin   
    backend(autocad) 
    delete_all_shapes()
    pav_slabs(u0(), 10, 7, 4, 3)
end

Expected result:
<img src="./Plots/slabs.png" width="400">

Update `pav_slabs` to include interior walls:

In [ ]:
pav_slabs_walls(p, length, height, width, n_floors, n_x, n_y)=
    let level_to_level_height = (height-2)/n_floors
        for i in division(0, height-2, n_floors, false)
            slab(rectangular_path(p, width, length), level=i, family=pav_slab_fam)
        end
        for i in division(0, height-2-level_to_level_height, n_floors-1, false)
            walls_x = [wall([p+vx(x), p+vxy(x, length)], bottom_level=i, top_level=level_to_level_height+i) 
                for x in division(width/n_x-1, width, n_x, false)]
            walls_y = [wall([p+vy(y), p+vxy(width, y)], bottom_level=i, top_level=level_to_level_height+i) 
                for y in division(length/n_y-1, length, n_y, false)]
        end
    end 

`floors` joins structural walls to the previous function: 

In [ ]:
function floors(p)
    let panel_w = (pav_length-2*d_length)/n_panels_y
        dist_x = pav_width-d_width*2
        dist_y_1 = (pav_length-d_length*2)/3
        dist_y_2 = (pav_length-d_length*2)*2/3
        dist_z = pav_height-d_height
    
    # structural walls
    wall([p+vxy(d_width, dist_y_1), p+vxy(pav_width-d_width, dist_y_1)], 
            top_level = level(dist_z), family=pav_wall_struct_fam)
    wall([p+vxy(d_width, dist_y_2), p+vxy(pav_width-d_width, dist_y_2)], 
            top_level = level(dist_z), family=pav_wall_struct_fam)
        
    # floor plans
    pav_slabs_walls(p+vxy(d_width, dist_y_1), dist_y_2+panel_w/2, dist_z, 
        dist_x, n_floors, n_wall_in_width, n_wall_in_length)
    slab(rectangular_path(p+vx(d_width), dist_x, dist_y_1), level=0, family=pav_slab_fam)

    end
end

In [ ]:
@test begin
    backend(autocad) 
    delete_all_shapes()
    floors(u0())
end

Expected result:
<img src="./Plots/slab_wall.png" width="600">

In [ ]:
@test begin
    backend(unity) 
    delete_all_shapes()
    set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/Grass"))
    ground()
    pav_walls(u0(), n_panels_x, n_panels_y, n_glass_verts)
    roof_surf(u0(), n_panels_x, n_panels_y)
    floors(u0())
    set_view(xyz(120.72909545898438,142.31788635253906,18.540668487548828), xyz(120.00056457519531,141.63906860351562,18.4487361907959), 35.0)
#     render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
#     render_view("interior_walls_unity")    
end

Expected result:
<img src="./Plots/interior_walls_unity.png" width="800">

## Site

Random tree definition:

In [ ]:
leaf(p, r, mat) =
    if current_backend() == unity
        with(current_material, get_material(mat)) do
            sphere(p, r)
        end
    else
        sphere(p, r)
    end


branch(pb, rb, pt, rt, mat)=
    if current_backend() == unity
        with(current_material, get_material(mat)) do
            cone_frustum(pb, rb, pt, rt)
        end
    else
        cone_frustum(pb, rb, pt, rt)
    end


tree(p_base, r_base, w_branch, fi, psi, min_f_w, max_f_w, max_fi, min_psi, max_psi) =
  let psi = psi > pi/2 ? psi - pi/2 : psi < -pi/2 ? psi + pi/2 : psi
      p_top = p_base + vsph(w_branch, fi, psi)
      r_top = r_base/2
      leaf_mat = "Default/Materials/Grass"
      branch_mat = "materials/wood/ExteriorWood9"
      branch(p_base, r_base, p_top, r_top, branch_mat)
      if w_branch < 0.01 || r_top < 0.001
          leaf(p_top, 0.05, leaf_mat)
      else
          tree(p_top, r_top, w_branch*random_range(min_f_w, max_f_w), 
            fi + random(max_fi), psi + random_range(min_psi, max_psi), 
            min_f_w, max_f_w, max_fi, min_psi, max_psi)
          tree(p_top, r_top, w_branch*random_range(min_f_w, max_f_w), 
            fi - random(max_fi), psi - random_range(min_psi, max_psi), 
            min_f_w, max_f_w, max_fi, min_psi, max_psi)
      end
  end


random_tree(p)=
  let r_base = random_range(0.05, 0.3)
      w_branch = random_range(2, 5)
      fi = 0
      psi = 0
      min_f_w = 0.6 # branch width max factor
      max_f_w = 0.9 # branch width min factor
      max_fi = 2pi
      min_psi = pi/16
      max_psi = pi/2
      tree(p, r_base, w_branch, fi, psi, min_f_w, max_f_w, max_fi, min_psi, max_psi)

end

In [ ]:
@test begin
    backend(unity) 
    delete_all_shapes()
    ground()
    map(n -> random_tree(x(n)), 0:2:20)
#     render_view("tress")
end

Expected result:
<img src="./Plots/trees.png" width="800">

## Backend specifics

Ground:

In [ ]:
ground_fam = slab_family_element(default_slab_family())
ground() =
    let x = 1000
        y = 1000
        z = -0.01
      slab(closed_polygonal_path([xyz(-x,y,z), xyz(x,y,z), xyz(x,-y,z), xyz(-x,-y,z)]), level(-0.05), ground_fam)
    end

BIM family materials for Unity backend:

In [ ]:
set_backend_family(default_truss_bar_family(), unity, unity_material_family("Default/Materials/Aluminum"))
set_backend_family(free_node_fam, unity, unity_material_family("Default/Materials/Aluminum"))
set_backend_family(sup_node_fam, unity, unity_material_family("materials/metal/Copper"))
set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/Grass")) # -- grass ground
# set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/White")) # -- white ground
# set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/WhiteUnlit")) # -- white ground no shadows
set_backend_family(default_panel_family(), unity, unity_material_family("Default/Materials/GlassBlue"))
set_backend_family(frame_fam, unity, unity_material_family("Default/Materials/Steel"))
set_backend_family(roof_panel_fam, unity, unity_material_family("Default/Materials/Aluminum"))
set_backend_family(yellow_panel_fam, unity, unity_material_family("materials/metal/YellowCopper"))
set_backend_family(pav_slab_fam, unity, unity_material_family("Default/Materials/Plaster"))

## Complete Building

The `GymPav` funtion gathers all building elements modeled, plus the random tree line in front of the west façade. Given a point `p`, the funtion creates the full building structure in acoordance with the parameters provided in `Pavilion Dimensions`.

In [ ]:
function GymPav(p)
    roof_surf(p, n_panels_x, n_panels_y)
#     roof_truss(p, 30, 70)
    pav_walls(p, n_panels_x, n_panels_y, n_glass_verts)
    floors(p)
    map(n -> random_tree(p+vxy(-5, n)), 0:2:pav_length) # tree line in the west façade
end

In [ ]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    GymPav(u0())
end

Saved render views:

In [ ]:
# set_view(xyz(-27.78546905517578,-12.493826866149902,2.73970365524292), xyz(-27.147165298461914,-11.735978126525879,2.874735116958618), 35.0)
# render_view("pav1")

# set_view(xyz(37.13166809082031,96.70274353027344,8.049731254577637), xyz(36.132999420166016,96.71927642822266,8.098581314086914), 35.0)
# render_view("pav2")

# set_view(xyz(86.96595001220703,87.33833312988281,7.179433822631836), xyz(85.97071075439453,87.31035614013672,7.27283239364624), 35.0)
# render_view("pav3")

# set_view(xyz(-22.71254539489746,144.94818115234375,7.690451622009277), xyz(-22.17136001586914,144.1073455810547,7.7015814781188965), 35.0)
# render_view("pav4")

# set_view(xyz(-12.342000961303711,103.7385025024414,3.020662307739258), xyz(-11.917301177978516,102.83597564697266,3.0918772220611572), 35.0)
# render_view("pav5")

# set_view(xyz(-2.6654765605926514,130.2740020751953,1.4012055397033691), xyz(-2.5774435997009277,129.29388427734375,1.5790280103683472), 35.0)
# render_view("pav6")

# set_view(xyz(126.3520736694336,-49.96141815185547,27.672582626342773), xyz(125.61244201660156,-49.31004333496094,27.503311157226562), 35.0)
# render_view("pav7")

__Unity renders__

| Pav 1 | Pav 2 |
|-------|-------|
| <img src="./Plots/pav1.png" width="450"> | <img src="./Plots/pav2.png" width="450"> |

| Pav 3 | Pav 4 |
|-------|-------|
| <img src="./Plots/pav3.png" width="450"> | <img src="./Plots/pav4.png" width="450"> |

| Pav 7 | Pav 6 |
|-------|-------|
| <img src="./Plots/pav7.png" width="450"> | <img src="./Plots/pav6.png" width="450"> |

| Pav 5 |
|-------|
| <img src="./Plots/pav5.png" width="900"> |

_The end_